<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_optimised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab

**Version 2.2 - 31 Dec 2025**

**Realistic expectations:**
- Startup: ~30 minutes (Google Drive I/O is slow)
- URL ready: ~10 minutes after startup completes
- Model loading: Additional time depending on model size

This is the nature of running from Google Drive. The T4 GPU is worth the wait if you need VRAM your local machine doesn't have.

**Setup:**
1. Runtime → Change runtime type → T4 GPU
2. Runtime → Run all
3. Wait for the cloudflared URL at the bottom

In [ ]:
#@title 0. Load Secrets (Optional)
import os

try:
    from google.colab import userdata
    secrets_found = []
    for key in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                secrets_found.append(key)
        except:
            pass
    if secrets_found:
        print(f"✓ Loaded: {', '.join(secrets_found)}")
    else:
        print("ℹ️  No secrets found")
except:
    print("ℹ️  Secrets not available")

In [ ]:
#@title 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

WORKSPACE = "/content/drive/MyDrive/ComfyUI"

import os
if not os.path.exists(WORKSPACE):
    raise RuntimeError(f"❌ ComfyUI not found at {WORKSPACE}")

print(f"✓ Workspace: {WORKSPACE}")

In [ ]:
#@title 2. Install Dependencies
import time
start = time.time()

%cd {WORKSPACE}

print("Installing core requirements...")
!pip install -q -r requirements.txt
!pip install -q --upgrade comfyui-frontend-package

print("\nInstalling custom node dependencies (this takes a while)...")
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

print(f"\n✓ Done in {(time.time()-start)/60:.1f} minutes")

In [ ]:
#@title 3. Verify GPU
import torch
print(f"PyTorch {torch.__version__}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("✓ GPU ready")
else:
    print("❌ No GPU! Runtime → Change runtime type → T4 GPU")

In [ ]:
#@title 4. Start ComfyUI
import subprocess, threading, time, socket, re

# Install cloudflared
![ ! -f ~/cloudflared.deb ] && wget -q -O ~/cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared.deb 2>/dev/null

url = None
start_time = time.time()

def tunnel(port=8188):
    global url
    # Wait for server
    for _ in range(300):
        try:
            s = socket.socket()
            s.settimeout(1)
            if s.connect_ex(('127.0.0.1', port)) == 0:
                s.close()
                break
        except:
            pass
        time.sleep(1)
    
    p = subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}'],
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    for line in p.stderr:
        if 'trycloudflare.com' in line:
            m = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if m:
                url = m.group(0)
                break

threading.Thread(target=tunnel, daemon=True).start()

%cd {WORKSPACE}
print("Starting ComfyUI...")
print("Custom nodes will take several minutes to load from Drive.\n")

p = subprocess.Popen(['python', 'main.py', '--dont-print-server'],
                     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

ready = False
for line in p.stdout:
    print(line, end='')
    
    if 'Import times for custom nodes:' in line and not ready:
        ready = True
        time.sleep(3)
        elapsed = (time.time() - start_time) / 60
        
        # Wait for URL
        for _ in range(30):
            if url:
                print(f"\n{'='*60}")
                print(f"🎉 READY after {elapsed:.1f} minutes")
                print(f"🌐 {url}")
                print(f"{'='*60}\n")
                break
            time.sleep(1)

## Why is it slow?

Google Drive is mounted via FUSE, which has ~100-500ms latency per file operation. Your `custom_nodes` folder has 18,841 files. Every Python import, every file read, every dependency check hits this latency.

**There's no fix for this** — it's fundamental to how Colab + Drive works.

## When is Colab worth it?

When you need more VRAM than your local GPU has:
- T4: 16 GB VRAM
- A100: 40 or 80 GB VRAM
- L4: 24 GB VRAM

For 65 GB Flux Dev at 4 megapixel, you'll need A100 80GB.